In [2]:
import numpy as np
import pickle
'''
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as utils
'''
from helpers import *

%load_ext autoreload
%autoreload 2

In [52]:
# define file paths

path_embeddings = 'pretrained_glove/embeddings200_pretrained_reduced.npy'
path_vocab = 'pretrained_glove/vocab_pretrained_reduced.pkl'
path_train_pos = 'pos_train.txt'
path_train_neg = 'neg_train.txt'
path_test = 'test_data.txt'

In [59]:
# load word embeddings
embeddings = np.load(path_embeddings)
# add line of zeroes to the embeddings for empty words
embeddings = np.append(embeddings, np.zeros((1, embeddings.shape[1])), axis=0)
# load vocabulary
with open(path_vocab, 'rb') as f:
    vocab = pickle.load(f)

In [15]:
# find maximal tweet length (number of words)
longest = 0
for file in [path_train_pos, path_train_neg, path_test]:
    with open(file) as f:
        for line in f:
            length = len(line.strip().split())
            if length > longest:
                longest = length
            
print("Longest tweet has {:d} words".format(longest))


Longest tweet has 64 words


In [62]:
x = []
y = []

with open(path_train_pos) as f:
    for line in f:
        tweet = embeddings.shape[0] * np.ones((longest)).astype(int)
        wordcount = 0
        y.append(np.asarray([1, 0]).astype(int))
        for word in line.strip().split():
            index = vocab.get(word, -1);
            # skip words for which we have no embedding
            if(index != -1):
                tweet[wordcount] = index
                wordcount += 1
        x.append(tweet)
        
with open(path_train_neg) as f:
    for line in f:
        tweet = embeddings.shape[0] * np.ones((longest)).astype(int)
        wordcount = 0
        y.append(np.asarray([0, 1]).astype(int))
        for word in line.strip().split():
            index = vocab.get(word, -1);
            # skip words for which we have no embedding
            if(index != -1):
                tweet[wordcount] = index
                wordcount += 1
        x.append(tweet)

x_train = np.asarray(x)
y_train = np.asarray(y)
 
# Shuffle tweets
x_train, y_train = shuffle(x_train, y_train)

In [63]:
x = []

with open(path_test) as f:
    for line in f:
        tweet = embeddings.shape[0] * np.ones((longest)).astype(int)
        wordcount = 0
        for word in line.strip().split():
            index = vocab.get(word, -1);
            # skip words for which we have no embedding
            if(index != -1):
                tweet[wordcount] = index
                wordcount += 1
        x.append(tweet)

x_submission = np.asarray(x)

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, n_input, embeddings):
        super.__init__()
        
        self.embeddings = torch.nn.Linear

In [9]:
# set aside a small portion for validation
testset = 1000

x_test = x_train[0:testset, :]
y_test = y_train[0:testset]
x_train_log = x_train[testset + 1:, :]
y_train_log = y_train[testset + 1:]

# standardize
#x_train_log, mean, std = standardize(x_train_log)

# add offset
x_train_log = add_offset(x_train_log)

# train using logistic regression (SGD)
initial_w = np.random.rand(x_train_log.shape[1])
epochs = 40
batch_size = 100
gamma = 0.0001
lambda_ = 0.01
print_every = int(50000 / batch_size)

weights, loss = reg_logistic_regression(y_train_log, x_train_log, initial_w, epochs, batch_size, gamma, lambda_, print_every)

# free up memory
del x_train_log
del y_train_log

epoch	 1 	loss:  80.41898758565523
epoch	 1 	loss:  72.63990867432156
epoch	 1 	loss:  72.27247042275462
epoch	 2 	loss:  71.36846290841906
epoch	 2 	loss:  70.9661952624341
epoch	 2 	loss:  70.56315594898248
epoch	 3 	loss:  70.12605256449098
epoch	 3 	loss:  69.6703350456351
epoch	 3 	loss:  69.63211465423336
epoch	 4 	loss:  69.31261762711519
epoch	 4 	loss:  69.08028647668529
epoch	 4 	loss:  68.99231228631497
epoch	 5 	loss:  68.82432824619028
epoch	 5 	loss:  68.72600853653859
epoch	 5 	loss:  68.63533220042841
epoch	 6 	loss:  68.48845100318616
epoch	 6 	loss:  68.6223296969754
epoch	 6 	loss:  68.3797046782826
epoch	 7 	loss:  68.36875349583184
epoch	 7 	loss:  68.43275071749726
epoch	 7 	loss:  68.34038315851258
epoch	 8 	loss:  68.28139345077676
epoch	 8 	loss:  68.31877622957936
epoch	 8 	loss:  68.2911010593855
epoch	 9 	loss:  68.20385412113002
epoch	 9 	loss:  68.29297923386456
epoch	 9 	loss:  68.1807471177709
epoch	 10 	loss:  68.21568128259491
epoch	 10 	loss:  68.2191

In [10]:
# standardize
#x_test_log = standardize_test(x_test, mean, std)

# add offset
x_test_log = add_offset(x_test)

y_pred = predict_logistic_labels(weights, x_test_log)
accuracy = get_accuracy(y_test, y_pred)
print(accuracy)

0.593


In [11]:
filename = 'logreg_preprocessing12_reduced.csv'

# standardize
#x_submission_log = standardize_test(x_submission_log, mean, std)

# add offset
x_submission_log = add_offset(x_submission)

y_submission = predict_logistic_labels(weights, x_submission_log)
ids = np.arange(len(y_submission)) + 1

create_csv_submission(ids, y_submission, filename)